In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

## Cargar datos


In [3]:
transacciones = pd.read_csv('../Datos/transacciones_con_features.csv', encoding='utf-8')
cotizaciones = pd.read_csv('../Datos/cotizaciones_con_features.csv', encoding='utf-8')

print(f"Transacciones: {transacciones.shape}")
print(f"Cotizaciones: {cotizaciones.shape}")


Transacciones: (2099287, 43)
Cotizaciones: (178378, 22)


## Convertir fechas

In [4]:
transacciones['fecha'] = pd.to_datetime(transacciones['fecha'])

print(f"Rango transacciones: {transacciones['fecha'].min()} a {transacciones['fecha'].max()}")


Rango transacciones: 1971-01-02 00:00:00 a 1973-01-31 00:00:00


# Dividir transacciones


In [5]:
fecha_corte = datetime(1973, 1, 1)

train_data = transacciones[transacciones['fecha'] < fecha_corte].copy()
test_data = transacciones[transacciones['fecha'] >= fecha_corte].copy()

print(f"Transacciones entrenamiento (1971-1972): {len(train_data):,}")
print(f"Transacciones validación (1973): {len(test_data):,}")


Transacciones entrenamiento (1971-1972): 2,015,795
Transacciones validación (1973): 83,492



## Analisis de Overlap

In [6]:
clientes_entrenamiento = set(train_data['id'].unique())
clientes_validacion = set(test_data['id'].unique())
clientes_comunes = clientes_entrenamiento.intersection(clientes_validacion)

productos_entrenamiento = set(train_data['producto'].unique())
productos_validacion = set(test_data['producto'].unique())
productos_comunes = productos_entrenamiento.intersection(productos_validacion)

print(f"Clientes comunes: {len(clientes_comunes):,} / {len(clientes_entrenamiento):,} ({len(clientes_comunes)/len(clientes_entrenamiento)*100:.1f}%)")
print(f"Productos comunes: {len(productos_comunes):,} / {len(productos_entrenamiento):,} ({len(productos_comunes)/len(productos_entrenamiento)*100:.1f}%)")


Clientes comunes: 11,330 / 404,601 (2.8%)
Productos comunes: 3,085 / 7,177 (43.0%)


## Guardar datos divididos

In [7]:
output_dir = '../Datos/validacion_historica'
os.makedirs(output_dir, exist_ok=True)
train_data.to_csv(f'{output_dir}/transacciones_entrenamiento.csv', index=False)
test_data.to_csv(f'{output_dir}/transacciones_validacion.csv', index=False)


## Importar Modelos

In [56]:
import os
import joblib
import pandas as pd
import numpy as np
from collections import defaultdict

# Obtener la ruta del directorio actual (Modelos)
MODELOS_DIR = "../Modelos"

# Cargar los modelos entrenados desde archivos .pkl
print("Cargando modelos entrenados...")

try:
    content_model = joblib.load(os.path.join(MODELOS_DIR, 'content_model.pkl'))
    print("✓ Modelo de contenido cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de contenido: {e}")
    content_model = None

try:
    copurchase_model = joblib.load(os.path.join(MODELOS_DIR, 'co_purchase_model.pkl'))
    print("✓ Modelo de co-compra cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de co-compra: {e}")
    copurchase_model = None

try:
    coquote_model = joblib.load(os.path.join(MODELOS_DIR, 'co_quotation_model.pkl'))
    print("✓ Modelo de co-cotización cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo de co-cotización: {e}")
    coquote_model = None

try:
    hybrid_model = joblib.load(os.path.join(MODELOS_DIR, 'hybrid_model.pkl'))
    print("✓ Modelo híbrido cargado")
except Exception as e:
    print(f"⚠️ Error cargando modelo híbrido: {e}")
    hybrid_model = None

print("Todos los modelos han sido cargados.\n")

# ================================================================
# FUNCIONES DE RECOMENDACIÓN PARA USAR LOS MODELOS CARGADOS
# ================================================================

def get_content_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en contenido usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'similarity_score']
    """
    if content_model is None:
        print("Error: Modelo de contenido no disponible")
        return pd.DataFrame()
    
    similarity_matrix = content_model['similarity_matrix']
    product_to_idx = content_model['product_to_idx']
    idx_to_product = content_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    recommendations = []
    for i, score in sim_scores:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'similarity_score': score})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_copurchase_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en co-compra usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'co_purchase_count']
    """
    if copurchase_model is None:
        #print("Error: Modelo de co-compra no disponible")
        return pd.DataFrame()
    
    co_occurrence_matrix = copurchase_model['co_occurrence_matrix']
    product_to_idx = copurchase_model['product_to_idx']
    idx_to_product = copurchase_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    co_buys = co_occurrence_matrix[idx, :]
    co_buy_indices = co_buys.indices
    co_buy_values = co_buys.data
    
    if len(co_buy_indices) == 0:
        #print(f"Producto '{input_product}' no tiene co-compras registradas")
        return pd.DataFrame()
    
    recommendations = []
    cobuy_pairs = sorted(zip(co_buy_indices, co_buy_values), key=lambda x: x[1], reverse=True)
    
    for i, count in cobuy_pairs:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'co_purchase_count': count})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_coquotation_recommendations(input_product, N=10):
    """
    Genera recomendaciones basadas en co-cotización usando el modelo cargado.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'co_quotation_count']
    """
    if coquote_model is None:
        #print("Error: Modelo de co-cotización no disponible")
        return pd.DataFrame()
    
    co_quotation_matrix = coquote_model['co_quotation_matrix']
    product_to_idx = coquote_model['product_to_idx']
    idx_to_product = coquote_model['idx_to_product']
    
    if input_product not in product_to_idx:
        print(f"Error: Producto '{input_product}' no encontrado")
        return pd.DataFrame()
    
    idx = product_to_idx[input_product]
    co_quotes = co_quotation_matrix[idx, :]
    co_quote_indices = co_quotes.indices
    co_quote_values = co_quotes.data
    
    if len(co_quote_indices) == 0:
        None
        #print(f"Producto '{input_product}' no tiene co-cotizaciones registradas")
        return pd.DataFrame()
    
    recommendations = []
    coquote_pairs = sorted(zip(co_quote_indices, co_quote_values), key=lambda x: x[1], reverse=True)
    
    for i, count in coquote_pairs:
        if i == idx:
            continue
        product_name = idx_to_product.get(i)
        if product_name and len(recommendations) < N:
            recommendations.append({'producto': product_name, 'co_quotation_count': count})
        elif len(recommendations) >= N:
            break
    
    return pd.DataFrame(recommendations)

def get_hybrid_recommendations(content_weight, cf_buy_weight, cf_quote_weight, input_product, N=10, k=2):
    """
    Genera recomendaciones híbridas usando re-ranking.
    
    Args:
        input_product (str): ID del producto
        N (int): Número de recomendaciones finales
        content_weight (float): Peso para contenido
        cf_buy_weight (float): Peso para co-compra
        cf_quote_weight (float): Peso para co-cotización
        k (int): Constante de suavizado
    
    Returns:
        pandas.DataFrame: Recomendaciones con columnas ['producto', 'hybrid_score']
    """
    if hybrid_model is None:
        print("Error: Modelo híbrido no disponible")
        return pd.DataFrame()
    
    # Obtener candidatos de cada método
    content_recs = get_content_recommendations(input_product, N=50)
    copurchase_recs = get_copurchase_recommendations(input_product, N=50)
    coquote_recs = get_coquotation_recommendations(input_product, N=50)
    
    content_weight = 0
    cf_buy_weight = 1
    cf_quote_weight = 0
    
    if copurchase_recs.empty:
        content_weight = 0
        cf_buy_weight = 0
        cf_quote_weight = 1
    
    if copurchase_recs.empty and coquote_recs.empty:
        content_weight = 1
        cf_buy_weight = 0
        cf_quote_weight = 0
        
    # Calcular scores híbridos
    hybrid_scores = defaultdict(float)
    
    # Contribución del modelo de contenido
    if not content_recs.empty:
        for rank, row in enumerate(content_recs.itertuples(), 1):
            score_contribution = content_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    # Contribución del modelo de co-compra
    if not copurchase_recs.empty:
        for rank, row in enumerate(copurchase_recs.itertuples(), 1):
            score_contribution = cf_buy_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    # Contribución del modelo de co-cotización
    if not coquote_recs.empty:
        for rank, row in enumerate(coquote_recs.itertuples(), 1):
            score_contribution = cf_quote_weight * (1.0 / (rank + k))
            hybrid_scores[row.producto] += score_contribution
    
    if not hybrid_scores:
        print(f"No se encontraron candidatos para '{input_product}'")
        return pd.DataFrame()
    
    # Crear DataFrame final
    final_recs = pd.DataFrame(hybrid_scores.items(), columns=['producto', 'hybrid_score'])
    final_recs = final_recs[final_recs['producto'] != input_product]
    final_recs = final_recs.sort_values('hybrid_score', ascending=False).head(N)
    
    return final_recs

# ================================================================
# FUNCIÓN DE PRUEBA PARA TODOS LOS MODELOS
# ================================================================




Cargando modelos entrenados...
✓ Modelo de contenido cargado
✓ Modelo de co-compra cargado
✓ Modelo de co-cotización cargado
✓ Modelo híbrido cargado
Todos los modelos han sido cargados.



## Funcion para probar todos los modelos

In [58]:
def test_all_models(peso_cont, peso_trans, peso_cot, test_product, N=5):
    """
    Prueba todos los modelos con un producto de ejemplo y devuelve los resultados.
    
    Args:
        test_product (str): Producto para probar
        N (int): Número de recomendaciones por modelo
    
    Returns:
        dict: Diccionario con las recomendaciones de los 4 modelos:
              {
                'content': DataFrame,
                'co_purchase': DataFrame, 
                'co_quotation': DataFrame,
                'hybrid': DataFrame,
                'input_product': str
              }
    """

    
    # Inicializar diccionario de resultados
    results = {
        'input_product': test_product,
        'content': pd.DataFrame(),
        'co_purchase': pd.DataFrame(),
        'co_quotation': pd.DataFrame(),
        'hybrid': pd.DataFrame()
    }
    
    # Probar modelo de contenido
    content_recs = get_content_recommendations(test_product, N=N)
    results['content'] = content_recs
    if not content_recs.empty:
        #print(content_recs)
        None
    else:
        None
        #print("No se encontraron recomendaciones de contenido")
    
    # Probar modelo de co-compra
    copurchase_recs = get_copurchase_recommendations(test_product, N=N)
    results['co_purchase'] = copurchase_recs
    if not copurchase_recs.empty:
        #print(copurchase_recs)
        None
    else:
        None
        #print("No se encontraron recomendaciones de co-compra")
    
    # Probar modelo de co-cotización
    coquote_recs = get_coquotation_recommendations(test_product, N=N)
    results['co_quotation'] = coquote_recs
    if not coquote_recs.empty:
        #print(coquote_recs)
        None
    else:
        None
        #print("No se encontraron recomendaciones de co-cotización")
    
    # Probar modelo híbrido
    hybrid_recs = get_hybrid_recommendations(peso_cont, peso_trans, peso_cot, test_product, N=N)
    results['hybrid'] = hybrid_recs
    if not hybrid_recs.empty:
        #print(hybrid_recs)
        None
    else:
        None
        #print("No se encontraron recomendaciones híbridas")
    

    
    # Resumen de productos únicos encontrados
    all_products = set()
    for model_name, df in results.items():
        if model_name != 'input_product' and not df.empty:
            all_products.update(df['producto'].tolist())

    return results

# Exportar todo
__all__ = [
    'content_model', 
    'copurchase_model', 
    'coquote_model', 
    'hybrid_model',
    'get_content_recommendations',
    'get_copurchase_recommendations',
    'get_coquotation_recommendations',
    'get_hybrid_recommendations',
    'test_all_models'
]
# Para usar la función, ejecuta: test_all_models("producto_ejemplo")


## Ejemplo de uso

In [32]:
producto = "producto_2"
recomendaciones = test_all_models(0.2, 0.5, 0.3, producto, N=5)
recomendaciones

{'input_product': 'producto_2',
 'content':         producto  similarity_score
 0  producto_1232          0.999869
 1  producto_1120          0.999325
 2  producto_1110          0.999110
 3  producto_3115          0.998920
 4   producto_837          0.998911,
 'co_purchase':         producto  co_purchase_count
 0  producto_3126                 17
 1   producto_413                 17
 2  producto_1120                 16
 3   producto_119                 16
 4   producto_110                 15,
 'co_quotation':         producto  co_quotation_count
 0  producto_3126                  13
 1  producto_1315                   6
 2   producto_208                   6
 3   producto_632                   6
 4   producto_642                   6,
 'hybrid':          producto  hybrid_score
 50  producto_3126      0.266667
 1   producto_1120      0.163636
 0   producto_1232      0.135714
 51   producto_413      0.125000
 19  producto_1315      0.117424}

## Preparar Datos de validación


In [33]:
# Crear diccionario de compras reales en período de validación
compras_reales = defaultdict(set)
for _, row in test_data.iterrows():
    compras_reales[row['id']].add(row['producto'])

print(f"Clientes únicos en validación: {len(compras_reales)}")

Clientes únicos en validación: 25950


In [19]:
compras_reales

defaultdict(set,
            {131101: {'producto_5362'},
             174413: {'producto_188',
              'producto_2001',
              'producto_2368',
              'producto_277',
              'producto_30',
              'producto_3514',
              'producto_3853',
              'producto_421',
              'producto_4496',
              'producto_451',
              'producto_452',
              'producto_471',
              'producto_571',
              'producto_5957'},
             20407: {'producto_176',
              'producto_238',
              'producto_2866',
              'producto_305',
              'producto_3199',
              'producto_3473',
              'producto_43',
              'producto_5',
              'producto_531',
              'producto_658',
              'producto_67',
              'producto_72'},
             177174: {'producto_1001',
              'producto_15',
              'producto_2254',
              'producto_242',
              

## Función para calcular métricas

In [78]:
def calcular_metricas(recomendaciones_dict, compras_reales_cliente, k=5):
    """
    Calcula métricas para un cliente específico
    """
    metricas = {}
    
    for modelo, recs_df in recomendaciones_dict.items():
        if modelo == 'input_product' or recs_df.empty:
            continue
            
        # Obtener top K recomendaciones
        productos_recomendados = set(recs_df.head(k)['producto'].tolist())
        productos_comprados = compras_reales_cliente
        
        # Calcular métricas
        hits = len(productos_recomendados.intersection(productos_comprados))
        
        # Hit Rate (HR@K): ¿Al menos 1 recomendación fue comprada?
        hit_rate = 1 if hits > 0 else 0
        
        # Precision@K: De las K recomendaciones, ¿cuántas fueron compradas?
        if len(productos_recomendados) > len(productos_comprados):
            precision = hits / len(productos_comprados) if len(productos_comprados) > 0 else 0
        else:
            precision = hits / len(productos_recomendados) if len(productos_recomendados) > 0 else 0
            
        # Recall@K: De los productos comprados, ¿cuántos fueron recomendados?
        recall = hits / len(productos_comprados) if productos_comprados else 0
        
        # F1-Score
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metricas[modelo] = {
            'hits': hits,
            'hit_rate': hit_rate,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'productos_recomendados': len(productos_recomendados),
            'productos_comprados': len(productos_comprados)
        }
    
    return metricas

## Función principal de evaluación

In [15]:
# CELDA 3.5: Función de debug para un producto específico
def debug_producto(producto_evaluar, k=5):
    """
    Evalúa un producto específico y muestra todo el proceso detallado
    """
    
    try:
        # 1. Generar recomendaciones
        recomendaciones = test_all_models(producto_evaluar, N=k)
        
        for modelo, recs_df in recomendaciones.items():
            if modelo != 'input_product' and not recs_df.empty:
                productos_rec = recs_df['producto'].tolist()
                print(f"  {modelo}: {productos_rec}")
            elif modelo != 'input_product':
                print(f"  {modelo}: Sin recomendaciones")
        
        # 2. Buscar clientes que compraron este producto en validación
        clientes_que_compraron = []
        for cliente, productos_comprados in compras_reales.items():
            if producto_evaluar in productos_comprados:
                clientes_que_compraron.append(cliente)
        
        print(f"   Clientes encontrados: {len(clientes_que_compraron)}")
        if len(clientes_que_compraron) > 0:
            print(f"   Primeros 5 clientes: {clientes_que_compraron[:5]}")
        
        if not clientes_que_compraron:
            print("⚠️  Este producto no tiene clientes en período de validación")
            print("   No se pueden calcular métricas para este producto")
            return None
        
        # 3. Evaluar cada cliente detalladamente
        
        todas_las_metricas = []
        
        for i, cliente in enumerate(clientes_que_compraron[:3]):  # Solo primeros 3 para no saturar
            print(f"\n👤 CLIENTE {i+1}: {cliente}")
            compras_cliente = compras_reales[cliente]
            print(f"📝 Productos comprados: {sorted(list(compras_cliente))}")
            print(f"📊 Total productos: {len(compras_cliente)}")
            
            # Calcular métricas para este cliente
            metricas_cliente = calcular_metricas(recomendaciones, compras_cliente, k)
            todas_las_metricas.append(metricas_cliente)
            
            for modelo, metricas in metricas_cliente.items():
                
                # Obtener productos recomendados
                recs_df = recomendaciones[modelo]
                productos_rec = set(recs_df.head(k)['producto'].tolist()) if not recs_df.empty else set()
                interseccion = productos_rec.intersection(compras_cliente)
                
                print(f"    Recomendados: {sorted(list(productos_rec))}")
                print(f"    Comprados:    {sorted(list(compras_cliente))}")
                print(f"    ✅ Intersección: {sorted(list(interseccion))}")
                print(f"    Hits: {metricas['hits']}")
                print(f"    Hit Rate: {metricas['hit_rate']}")
                print(f"    Precision: {metricas['precision']:.3f}")
                print(f"    Recall: {metricas['recall']:.3f}")
                print(f"    F1-Score: {metricas['f1_score']:.3f}")
        
        if len(clientes_que_compraron) > 3:
            print(f"\n... y {len(clientes_que_compraron) - 3} clientes más")
        
        # 4. Calcular métricas promedio para este producto
        print(f"\n📊 MÉTRICAS PROMEDIO PARA PRODUCTO '{producto_evaluar}':")
        print("-" * 60)
        
        # Acumular todas las métricas
        metricas_acumuladas = defaultdict(list)
        for metricas_cliente in todas_las_metricas:
            for modelo, metricas in metricas_cliente.items():
                for metrica, valor in metricas.items():
                    metricas_acumuladas[f"{modelo}_{metrica}"].append(valor)
        
        # Calcular promedios
        modelos = ['content', 'co_purchase', 'co_quotation', 'hybrid']
        for modelo in modelos:
            if f"{modelo}_hit_rate" in metricas_acumuladas:
                print(f"\n🔹 {modelo.upper()}:")
                print(f"  Hit Rate promedio:  {np.mean(metricas_acumuladas[f'{modelo}_hit_rate']):.3f}")
                print(f"  Precision promedio: {np.mean(metricas_acumuladas[f'{modelo}_precision']):.3f}")
                print(f"  Recall promedio:    {np.mean(metricas_acumuladas[f'{modelo}_recall']):.3f}")
                print(f"  F1-Score promedio:  {np.mean(metricas_acumuladas[f'{modelo}_f1_score']):.3f}")
                print(f"  Evaluaciones:       {len(metricas_acumuladas[f'{modelo}_hit_rate'])}")
        
        print("\n" + "="*80)
        print("✅ Evaluación detallada completada!")
        
        return metricas_acumuladas
        
    except Exception as e:
        print(f"❌ Error evaluando producto {producto_evaluar}: {e}")
        return None

In [57]:
def evaluar_algoritmo(peso_cont, peso_trans, peso_cot, productos_test, n_productos_evaluar=100, k=10):
    """
    Evalúa el algoritmo con una muestra de productos
    """
    print(f"Evaluando {n_productos_evaluar} productos con top-{k} recomendaciones...")
    
    # Tomar muestra de productos para evaluar
    productos_evaluar = list(productos_test)[:n_productos_evaluar]
    
    # Diccionarios para acumular métricas
    metricas_totales = defaultdict(list)
    productos_evaluados = 0
    productos_sin_clientes = 0
    errores = 0
    
    
    for i, producto in enumerate(productos_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(productos_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(productos_evaluar)} ({(i+1)/len(productos_evaluar)*100:.1f}%)")
        
        try:
            # Generar recomendaciones para este producto
            recomendaciones = test_all_models(peso_cont, peso_trans, peso_cot, producto, N=k)
            
            # Buscar clientes que compraron este producto en validación
            clientes_que_compraron = []
            for cliente, productos_comprados in compras_reales.items():
                if producto in productos_comprados:
                    clientes_que_compraron.append(cliente)
            
            if not clientes_que_compraron:
                productos_sin_clientes += 1
                continue
                
            # Evaluar para cada cliente que compró este producto
            for cliente in clientes_que_compraron:
                compras_cliente = compras_reales[cliente]
                metricas_cliente = calcular_metricas(recomendaciones, compras_cliente, k)
                
                # Acumular métricas
                for modelo, metricas in metricas_cliente.items():
                    for metrica, valor in metricas.items():
                        metricas_totales[f"{modelo}_{metrica}"].append(valor)
            
            productos_evaluados += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:  # Solo mostrar primeros 5 errores
                print(f"Error evaluando producto {producto}: {e}")
            continue
    
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"  ✅ Productos evaluados: {productos_evaluados}")
    print(f"  ⚠️  Sin clientes en validación: {productos_sin_clientes}")
    print(f"  ❌ Errores: {errores}")
    
    return metricas_totales, productos_evaluados

## Ejecutar evaluación

In [79]:
productos_test = set(test_data['producto'].unique())
productos_train = set(train_data['producto'].unique())
productos_comunes = productos_test.intersection(productos_train)

resultados, n_evaluados = evaluar_algoritmo(0, 0, 0 ,productos_comunes, n_productos_evaluar=len(productos_comunes), k=5)

print(f"\n✅ Productos evaluados exitosamente: {n_evaluados} / {len(productos_comunes)}")
print(f"📊 Evaluaciones totales realizadas: {len(resultados.get('content_hit_rate', []))}")

# CELDA 6: Calcular métricas promedio
def calcular_metricas_promedio(resultados):
    """
    Calcula métricas promedio por modelo
    """
    modelos = ['content', 'co_purchase', 'co_quotation', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in resultados:
            metricas_finales[modelo] = {
                'hit_rate': np.mean(resultados[f"{modelo}_hit_rate"]),
                'precision': np.mean(resultados[f"{modelo}_precision"]),
                'recall': np.mean(resultados[f"{modelo}_recall"]),
                'f1_score': np.mean(resultados[f"{modelo}_f1_score"]),
                'n_evaluaciones': len(resultados[f"{modelo}_hit_rate"])
            }
    
    return metricas_finales

metricas_finales = calcular_metricas_promedio(resultados)



Evaluando 3085 productos con top-5 recomendaciones...
Progreso: 1/3085 (0.0%)
Progreso: 309/3085 (10.0%)
Progreso: 617/3085 (20.0%)
Progreso: 925/3085 (30.0%)
Progreso: 1233/3085 (40.0%)
Progreso: 1541/3085 (50.0%)
Progreso: 1849/3085 (59.9%)
Progreso: 2157/3085 (69.9%)
Progreso: 2465/3085 (79.9%)
Progreso: 2773/3085 (89.9%)
Progreso: 3081/3085 (99.9%)

📈 ESTADÍSTICAS FINALES:
  ✅ Productos evaluados: 3085
  ⚠️  Sin clientes en validación: 0
  ❌ Errores: 0

✅ Productos evaluados exitosamente: 3085 / 3085
📊 Evaluaciones totales realizadas: 77738


In [81]:
metricas_finales

# Convertir a DataFrame para visualización
df_metricas = pd.DataFrame(metricas_finales).T

# Redondear valores numéricos para mejor visualización
df_metricas = df_metricas.round(4)

# Reordenar columnas para mejor presentación
df_metricas = df_metricas[['hit_rate', 'precision', 'recall', 'f1_score', 'n_evaluaciones']]

df_metricas



,hit_rate,precision,recall,f1_score,n_evaluaciones
content,0.0864,0.0223,0.0146,0.0166,77738.0
co_purchase,0.4698,0.1523,0.1083,0.1206,77694.0
co_quotation,0.4129,0.1316,0.0856,0.0975,22083.0
hybrid,0.4695,0.1519,0.1082,0.1205,77738.0


In [25]:
debug_producto("producto_4")

  content: ['producto_1468', 'producto_1220', 'producto_1188', 'producto_1447', 'producto_4691']
  co_purchase: ['producto_43', 'producto_27', 'producto_28', 'producto_19', 'producto_1355']
  co_quotation: Sin recomendaciones
  hybrid: ['producto_1468', 'producto_1220', 'producto_1188', 'producto_1447', 'producto_4691']
   Clientes encontrados: 30
   Primeros 5 clientes: [177250, 45094, 177643, 178000, 126123]

👤 CLIENTE 1: 177250
📝 Productos comprados: ['producto_268', 'producto_3097', 'producto_3271', 'producto_4', 'producto_8']
📊 Total productos: 5
    Recomendados: ['producto_1188', 'producto_1220', 'producto_1447', 'producto_1468', 'producto_4691']
    Comprados:    ['producto_268', 'producto_3097', 'producto_3271', 'producto_4', 'producto_8']
    ✅ Intersección: []
    Hits: 0
    Hit Rate: 0
    Precision: 0.000
    Recall: 0.000
    F1-Score: 0.000
    Recomendados: ['producto_1355', 'producto_19', 'producto_27', 'producto_28', 'producto_43']
    Comprados:    ['producto_268', 

defaultdict(list,
            {'content_hits': [0, 0, 0],
             'content_hit_rate': [0, 0, 0],
             'content_precision': [0.0, 0.0, 0.0],
             'content_recall': [0.0, 0.0, 0.0],
             'content_f1_score': [0, 0, 0],
             'content_productos_recomendados': [5, 5, 5],
             'content_productos_comprados': [5, 1, 1],
             'co_purchase_hits': [0, 0, 0],
             'co_purchase_hit_rate': [0, 0, 0],
             'co_purchase_precision': [0.0, 0.0, 0.0],
             'co_purchase_recall': [0.0, 0.0, 0.0],
             'co_purchase_f1_score': [0, 0, 0],
             'co_purchase_productos_recomendados': [5, 5, 5],
             'co_purchase_productos_comprados': [5, 1, 1],
             'hybrid_hits': [0, 0, 0],
             'hybrid_hit_rate': [0, 0, 0],
             'hybrid_precision': [0.0, 0.0, 0.0],
             'hybrid_recall': [0.0, 0.0, 0.0],
             'hybrid_f1_score': [0, 0, 0],
             'hybrid_productos_recomendados': [5, 5,

##  Evaluación por transacción multi-producto

In [68]:

def evaluar_ventas_cruzadas_por_transaccion(peso_cont, peso_trans, peso_cot, n_transacciones_evaluar=500, k=10):
    """
    Evalúa si el algoritmo puede predecir productos adicionales en transacciones multi-producto.
    Dado el primer producto de una transacción, ¿recomienda los otros productos comprados?
    
    Args:
        peso_cont, peso_trans, peso_cot: Pesos para el modelo híbrido
        n_transacciones_evaluar: Número de transacciones a evaluar
        k: Top-K recomendaciones a considerar
    
    Returns:
        dict: Métricas de evaluación por transacción
    """

    # 1. Encontrar transacciones multi-producto en datos de validación
    print("\n Analizando transacciones multi-producto en período de validación...")
    
    # Agrupar por cliente y fecha para identificar transacciones
    test_data['fecha'] = pd.to_datetime(test_data['fecha'])
    transacciones_grupo = test_data.groupby(['id', 'fecha'])
    
    # Filtrar solo transacciones con más de 1 producto
    transacciones_multi = []
    for (cliente, fecha), grupo in transacciones_grupo:
        productos = grupo['producto'].unique()
        if len(productos) >= 2:  # Transacciones con 2+ productos
            transacciones_multi.append({
                'cliente': cliente,
                'fecha': fecha,
                'productos': list(productos),
                'n_productos': len(productos)
            })
    
    print(f"📈 Transacciones multi-producto encontradas: {len(transacciones_multi)}")
    
    if len(transacciones_multi) == 0:
        print("❌ No se encontraron transacciones multi-producto")
        return {}
    
    # Estadísticas de transacciones multi-producto
    tamaños = [t['n_productos'] for t in transacciones_multi]
    print(f"📊 Estadísticas de tamaño de transacciones:")
    print(f"   - Promedio productos por transacción: {np.mean(tamaños):.2f}")
    print(f"   - Mediana: {np.median(tamaños):.0f}")
    print(f"   - Máximo: {max(tamaños)}")
    print(f"   - Mínimo: {min(tamaños)}")
    
    # 2. Tomar muestra de transacciones para evaluar
    n_evaluar = min(n_transacciones_evaluar, len(transacciones_multi))
    transacciones_evaluar = transacciones_multi[:n_evaluar]
    
    print(f"\n🎯 Evaluando {n_evaluar} transacciones...")
    print("Metodología: Dado el primer producto → ¿Se recomiendan los otros productos de la transacción?")
    
    # 3. Evaluar cada transacción
    metricas_transacciones = defaultdict(list)
    transacciones_evaluadas = 0
    errores = 0
    
    # Para debug: mostrar detalles de las primeras 3 transacciones
    mostrar_debug = True
    
    for i, transaccion in enumerate(transacciones_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(transacciones_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(transacciones_evaluar)} ({(i+1)/len(transacciones_evaluar)*100:.1f}%)")
        
        try:
            productos = transaccion['productos']
            producto_input = productos[0]  # Primer producto como input
            productos_target = set(productos[1:])  # Resto como target
            
            # DEBUG: Mostrar detalles de las primeras 3 transacciones
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n" + "="*60)
                print(f"🔍 DEBUG TRANSACCIÓN {transacciones_evaluadas + 1}")
                print(f"📅 Cliente: {transaccion['cliente']}, Fecha: {transaccion['fecha'].strftime('%Y-%m-%d')}")
                print(f"🛒 Productos en transacción: {productos}")
                print(f"🎯 Input: '{producto_input}' → Target: {sorted(list(productos_target))}")
            
            # Generar recomendaciones para el primer producto
            recomendaciones = test_all_models(peso_cont, peso_trans, peso_cot, producto_input, N=k)
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n📋 Recomendaciones generadas para '{producto_input}':")
                for modelo, recs_df in recomendaciones.items():
                    if modelo != 'input_product' and not recs_df.empty:
                        productos_rec = recs_df['producto'].tolist()
                        print(f"  {modelo}: {productos_rec}")
                    elif modelo != 'input_product':
                        print(f"  {modelo}: Sin recomendaciones")
            
            # Evaluar cada modelo
            for modelo, recs_df in recomendaciones.items():
                if modelo == 'input_product' or recs_df.empty:
                    continue
                
                # Productos recomendados (top-K)
                productos_recomendados = set(recs_df.head(k)['producto'].tolist())
                
                # Calcular intersección
                productos_acertados = productos_recomendados.intersection(productos_target)
                
                # Métricas
                hit_rate = 1 if len(productos_acertados) > 0 else 0
                precision = len(productos_acertados) / len(productos_recomendados) if productos_recomendados else 0
                recall = len(productos_acertados) / len(productos_target) if productos_target else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                
                # Guardar métricas
                metricas_transacciones[f"{modelo}_hit_rate"].append(hit_rate)
                metricas_transacciones[f"{modelo}_precision"].append(precision)
                metricas_transacciones[f"{modelo}_recall"].append(recall)
                metricas_transacciones[f"{modelo}_f1_score"].append(f1_score)
                metricas_transacciones[f"{modelo}_productos_acertados"].append(len(productos_acertados))
                metricas_transacciones[f"{modelo}_productos_target"].append(len(productos_target))
                
                # DEBUG: Mostrar métricas detalladas
                if mostrar_debug and transacciones_evaluadas < 3:
                    print(f"\n  🔹 {modelo.upper()}:")
                    print(f"    Recomendados: {sorted(list(productos_recomendados))}")
                    print(f"    Target:       {sorted(list(productos_target))}")
                    print(f"    ✅ Acertados: {sorted(list(productos_acertados))}")
                    print(f"    Hit Rate:  {hit_rate}")
                    print(f"    Precision: {precision:.3f}")
                    print(f"    Recall:    {recall:.3f}")
                    print(f"    F1-Score:  {f1_score:.3f}")
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print("="*60)
            
            transacciones_evaluadas += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando transacción {i}: {e}")
            continue
    
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"  ✅ Transacciones evaluadas: {transacciones_evaluadas}")
    print(f"  ❌ Errores: {errores}")
    
    # 4. Calcular métricas promedio
    print(f"\n📊 RESULTADOS DE VENTAS CRUZADAS POR TRANSACCIÓN:")
    print("="*60)
    
    modelos = ['content', 'co_purchase','co_quotation', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in metricas_transacciones:
            hit_rate_avg = np.mean(metricas_transacciones[f"{modelo}_hit_rate"])
            precision_avg = np.mean(metricas_transacciones[f"{modelo}_precision"])
            recall_avg = np.mean(metricas_transacciones[f"{modelo}_recall"])
            f1_avg = np.mean(metricas_transacciones[f"{modelo}_f1_score"])
            
            metricas_finales[modelo] = {
                'hit_rate': hit_rate_avg,
                'precision': precision_avg,
                'recall': recall_avg,
                'f1_score': f1_avg,
                'n_transacciones': len(metricas_transacciones[f"{modelo}_hit_rate"])
            }
            
            print(f"\n🔹 {modelo.upper()}:")
            print(f"  Hit Rate@{k}:     {hit_rate_avg:.3f} ({hit_rate_avg*100:.1f}% de transacciones con al menos 1 acierto)")
            print(f"  Precision@{k}:    {precision_avg:.3f}")
            print(f"  Recall@{k}:       {recall_avg:.3f}")
            print(f"  F1-Score:         {f1_avg:.3f}")
            print(f"  Transacciones:    {len(metricas_transacciones[f'{modelo}_hit_rate'])}")
    
    return metricas_finales, metricas_transacciones


In [82]:
# EJECUTAR EVALUACIÓN DE VENTAS CRUZADAS

# Usar pesos ejemplo - ajusta según tus necesidades
peso_contenido = 0.3
peso_transacciones = 0.4  
peso_cotizaciones = 0.3

resultados_ventas_cruzadas, metricas_detalladas = evaluar_ventas_cruzadas_por_transaccion(
    peso_contenido, peso_transacciones, peso_cotizaciones, 
    n_transacciones_evaluar=9999999, 
    k=10
)

# Guardar resultados
output_file_vc = '../Datos/validacion_historica/metricas_ventas_cruzadas.csv'
if resultados_ventas_cruzadas:
    resultados_df = pd.DataFrame(resultados_ventas_cruzadas).T
    resultados_df.to_csv(output_file_vc)
    print(f"\n💾 Resultados guardados en: {output_file_vc}")


📊 Analizando transacciones multi-producto en período de validación...
📈 Transacciones multi-producto encontradas: 16803
📊 Estadísticas de tamaño de transacciones:
   - Promedio productos por pedido: 3.74
   - Mediana: 3
   - Máximo: 34
   - Mínimo: 2
📋 Distribución de tamaños de pedido:
   - 2 productos: 6681 pedidos
   - 3 productos: 4275 pedidos
   - 4 productos: 1976 pedidos
   - 5 productos: 1189 pedidos
   - 6 productos: 785 pedidos
   - 7 productos: 513 pedidos
   - 8 productos: 384 pedidos
   - 9 productos: 282 pedidos
   - 10 productos: 215 pedidos
   - 11 productos: 146 pedidos
   - ... y 19 tamaños más

🎯 Evaluando 16803 transacciones...
Metodología: Dado el primer producto del pedido → ¿Se recomiendan los otros productos del mismo pedido?
Progreso: 1/16803 (0.0%)

🔍 DEBUG TRANSACCIÓN 1
🆔 Pedido: 93178
👤 Cliente: 3357
📅 Fecha: 1973-01-29 00:00:00
🛒 Productos en pedido: ['producto_3206', 'producto_4933', 'producto_843']
🎯 Input: 'producto_3206' → Target: ['producto_4933', 'pr

In [70]:
# CELDA NUEVA: Evaluación por transacción multi-producto (CORREGIDA)
def evaluar_ventas_cruzadas_por_transaccion(peso_cont, peso_trans, peso_cot, n_transacciones_evaluar=500, k=5):
    """
    Evalúa si el algoritmo puede predecir productos adicionales en transacciones multi-producto.
    Dado el primer producto de una transacción, ¿recomienda los otros productos comprados?
    
    Args:
        peso_cont, peso_trans, peso_cot: Pesos para el modelo híbrido
        n_transacciones_evaluar: Número de transacciones a evaluar
        k: Top-K recomendaciones a considerar
    
    Returns:
        dict: Métricas de evaluación por transacción
    """

    # 1. Encontrar transacciones multi-producto usando la columna 'pedido'
    print("\n📊 Analizando transacciones multi-producto en período de validación...")
    
    # Agrupar por pedido para identificar transacciones
    transacciones_grupo = test_data.groupby('pedido')
    
    # Filtrar solo transacciones con más de 1 producto
    transacciones_multi = []
    for pedido_id, grupo in transacciones_grupo:
        productos = grupo['producto'].unique()
        if len(productos) >= 2:  # Transacciones con 2+ productos
            transacciones_multi.append({
                'pedido_id': pedido_id,
                'cliente': grupo['id'].iloc[0],  # Cliente del pedido
                'fecha': grupo['fecha'].iloc[0] if 'fecha' in grupo.columns else 'N/A',
                'productos': list(productos),
                'n_productos': len(productos)
            })
    
    print(f"📈 Transacciones multi-producto encontradas: {len(transacciones_multi)}")
    
    if len(transacciones_multi) == 0:
        print("❌ No se encontraron transacciones multi-producto")
        return {}
    
    # Estadísticas de transacciones multi-producto
    tamaños = [t['n_productos'] for t in transacciones_multi]
    print(f"📊 Estadísticas de tamaño de transacciones:")
    print(f"   - Promedio productos por pedido: {np.mean(tamaños):.2f}")
    print(f"   - Mediana: {np.median(tamaños):.0f}")
    print(f"   - Máximo: {max(tamaños)}")
    print(f"   - Mínimo: {min(tamaños)}")
    
    # Distribución de tamaños
    from collections import Counter
    distribucion = Counter(tamaños)
    print(f"📋 Distribución de tamaños de pedido:")
    for tam in sorted(distribucion.keys())[:10]:  # Mostrar primeros 10
        print(f"   - {tam} productos: {distribucion[tam]} pedidos")
    if len(distribucion) > 10:
        print(f"   - ... y {len(distribucion) - 10} tamaños más")
    
    # 2. Tomar muestra de transacciones para evaluar
    n_evaluar = min(n_transacciones_evaluar, len(transacciones_multi))
    transacciones_evaluar = transacciones_multi[:n_evaluar]
    
    print(f"\n🎯 Evaluando {n_evaluar} transacciones...")
    print("Metodología: Dado el primer producto del pedido → ¿Se recomiendan los otros productos del mismo pedido?")
    
    # 3. Evaluar cada transacción
    metricas_transacciones = defaultdict(list)
    transacciones_evaluadas = 0
    errores = 0
    
    # Para debug: mostrar detalles de las primeras 3 transacciones
    mostrar_debug = True
    
    for i, transaccion in enumerate(transacciones_evaluar):
        # Mostrar progreso cada 10%
        if i % max(1, len(transacciones_evaluar) // 10) == 0:
            print(f"Progreso: {i+1}/{len(transacciones_evaluar)} ({(i+1)/len(transacciones_evaluar)*100:.1f}%)")
        
        try:
            productos = transaccion['productos']
            producto_input = productos[0]  # Primer producto como input
            productos_target = set(productos[1:])  # Resto como target
            
            # DEBUG: Mostrar detalles de las primeras 3 transacciones
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n" + "="*60)
                print(f"🔍 DEBUG TRANSACCIÓN {transacciones_evaluadas + 1}")
                print(f"🆔 Pedido: {transaccion['pedido_id']}")
                print(f"👤 Cliente: {transaccion['cliente']}")
                print(f"📅 Fecha: {transaccion['fecha']}")
                print(f"🛒 Productos en pedido: {productos}")
                print(f"🎯 Input: '{producto_input}' → Target: {sorted(list(productos_target))}")
            
            # Generar recomendaciones para el primer producto
            recomendaciones = test_all_models(peso_cont, peso_trans, peso_cot, producto_input, N=k)
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print(f"\n📋 Recomendaciones generadas para '{producto_input}':")
                for modelo, recs_df in recomendaciones.items():
                    if modelo != 'input_product' and not recs_df.empty:
                        productos_rec = recs_df['producto'].tolist()
                        print(f"  {modelo}: {productos_rec}")
                    elif modelo != 'input_product':
                        print(f"  {modelo}: Sin recomendaciones")
            
            # Evaluar cada modelo
            for modelo, recs_df in recomendaciones.items():
                if modelo == 'input_product' or recs_df.empty:
                    continue
                
                # Productos recomendados (top-K)
                productos_recomendados = set(recs_df.head(k)['producto'].tolist())
                
                # Calcular intersección
                productos_acertados = productos_recomendados.intersection(productos_target)
                
                # Métricas
                hit_rate = 1 if len(productos_acertados) > 0 else 0
                precision = len(productos_acertados) / len(productos_recomendados) if productos_recomendados else 0
                recall = len(productos_acertados) / len(productos_target) if productos_target else 0
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                
                # Guardar métricas
                metricas_transacciones[f"{modelo}_hit_rate"].append(hit_rate)
                metricas_transacciones[f"{modelo}_precision"].append(precision)
                metricas_transacciones[f"{modelo}_recall"].append(recall)
                metricas_transacciones[f"{modelo}_f1_score"].append(f1_score)
                metricas_transacciones[f"{modelo}_productos_acertados"].append(len(productos_acertados))
                metricas_transacciones[f"{modelo}_productos_target"].append(len(productos_target))
                
                # DEBUG: Mostrar métricas detalladas
                if mostrar_debug and transacciones_evaluadas < 3:
                    print(f"\n  🔹 {modelo.upper()}:")
                    print(f"    Recomendados: {sorted(list(productos_recomendados))}")
                    print(f"    Target:       {sorted(list(productos_target))}")
                    print(f"    ✅ Acertados: {sorted(list(productos_acertados))}")
                    print(f"    Hit Rate:  {hit_rate}")
                    print(f"    Precision: {precision:.3f}")
                    print(f"    Recall:    {recall:.3f}")
                    print(f"    F1-Score:  {f1_score:.3f}")
            
            if mostrar_debug and transacciones_evaluadas < 3:
                print("="*60)
            
            transacciones_evaluadas += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando transacción {i}: {e}")
            continue
    
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"  ✅ Transacciones evaluadas: {transacciones_evaluadas}")
    print(f"  ❌ Errores: {errores}")
    
    # 4. Calcular métricas promedio
    print(f"\n📊 RESULTADOS DE VENTAS CRUZADAS POR TRANSACCIÓN:")
    print("="*60)
    
    modelos = ['content', 'co_purchase', 'co_quotation', 'hybrid']
    metricas_finales = {}
    
    for modelo in modelos:
        if f"{modelo}_hit_rate" in metricas_transacciones:
            hit_rate_avg = np.mean(metricas_transacciones[f"{modelo}_hit_rate"])
            precision_avg = np.mean(metricas_transacciones[f"{modelo}_precision"])
            recall_avg = np.mean(metricas_transacciones[f"{modelo}_recall"])
            f1_avg = np.mean(metricas_transacciones[f"{modelo}_f1_score"])
            
            metricas_finales[modelo] = {
                'hit_rate': hit_rate_avg,
                'precision': precision_avg,
                'recall': recall_avg,
                'f1_score': f1_avg,
                'n_transacciones': len(metricas_transacciones[f"{modelo}_hit_rate"])
            }
            
            print(f"\n🔹 {modelo.upper()}:")
            print(f"  Hit Rate@{k}:     {hit_rate_avg:.3f} ({hit_rate_avg*100:.1f}% de pedidos con al menos 1 acierto)")
            print(f"  Precision@{k}:    {precision_avg:.3f}")
            print(f"  Recall@{k}:       {recall_avg:.3f}")
            print(f"  F1-Score:         {f1_avg:.3f}")
            print(f"  Pedidos evaluados: {len(metricas_transacciones[f'{modelo}_hit_rate'])}")
    
    return metricas_finales, metricas_transacciones



In [83]:
# CONTAR TODAS LAS TRANSACCIONES MULTI-PRODUCTO
print("📊 Contando todas las transacciones multi-producto en datos de test...")

transacciones_grupo = test_data.groupby('pedido')
transacciones_multi_total = 0

for pedido_id, grupo in transacciones_grupo:
    productos = grupo['producto'].unique()
    if len(productos) >= 2:
        transacciones_multi_total += 1

print(f"📈 Total de pedidos multi-producto a evaluar: {transacciones_multi_total}")

# EJECUTAR CON TODOS LOS DATOS
print("\n🚀 INICIANDO EVALUACIÓN DE VENTAS CRUZADAS CON TODOS LOS PEDIDOS MULTI-PRODUCTO...")
print("⚠️  Esto puede tomar mucho tiempo dependiendo del número de pedidos...")

# Usar pesos ejemplo - ajusta según tus necesidades
peso_contenido = 0.3
peso_transacciones = 0.4  
peso_cotizaciones = 0.3

resultados_ventas_cruzadas_completo, metricas_detalladas_completo = evaluar_ventas_cruzadas_por_transaccion(
    peso_contenido, peso_transacciones, peso_cotizaciones, 
    n_transacciones_evaluar=transacciones_multi_total,  # TODOS los pedidos multi-producto
    k=10
)

# Guardar resultados completos
output_file_vc_completo = '../Datos/validacion_historica/metricas_ventas_cruzadas_completo_pedido.csv'
if resultados_ventas_cruzadas_completo:
    resultados_df_completo = pd.DataFrame(resultados_ventas_cruzadas_completo).T
    resultados_df_completo.to_csv(output_file_vc_completo)
    print(f"\n💾 Resultados completos guardados en: {output_file_vc_completo}")

print("\n✅ EVALUACIÓN COMPLETA DE VENTAS CRUZADAS TERMINADA!")

📊 Contando todas las transacciones multi-producto en datos de test...
📈 Total de pedidos multi-producto a evaluar: 16803

🚀 INICIANDO EVALUACIÓN DE VENTAS CRUZADAS CON TODOS LOS PEDIDOS MULTI-PRODUCTO...
⚠️  Esto puede tomar mucho tiempo dependiendo del número de pedidos...

📊 Analizando transacciones multi-producto en período de validación...
📈 Transacciones multi-producto encontradas: 16803
📊 Estadísticas de tamaño de transacciones:
   - Promedio productos por pedido: 3.74
   - Mediana: 3
   - Máximo: 34
   - Mínimo: 2
📋 Distribución de tamaños de pedido:
   - 2 productos: 6681 pedidos
   - 3 productos: 4275 pedidos
   - 4 productos: 1976 pedidos
   - 5 productos: 1189 pedidos
   - 6 productos: 785 pedidos
   - 7 productos: 513 pedidos
   - 8 productos: 384 pedidos
   - 9 productos: 282 pedidos
   - 10 productos: 215 pedidos
   - 11 productos: 146 pedidos
   - ... y 19 tamaños más

🎯 Evaluando 16803 transacciones...
Metodología: Dado el primer producto del pedido → ¿Se recomiendan lo